In [17]:
import numpy as np
import pandas as pd
import fastf1
import os
import warnings
import xlsxwriter
warnings.simplefilter(action='ignore', category=FutureWarning) #ignore future warnings so our file is not polluted with unnecessary info.

setting local cache folder - not needed, but execution is faster

In [18]:
# Check if the subfolder exists
if not os.path.exists(os.getenv('LOCALAPPDATA') + "/pip/cache/fastF1"):
  # Create the subfolder
  os.makedirs(os.getenv('LOCALAPPDATA') + "/pip/cache/fastF1")

fastf1.Cache.enable_cache(os.getenv('LOCALAPPDATA') + "/pip/cache/fastF1")

In [19]:
# Define the path of the subfolder
telemetries_path = os.getcwd() + "/Telemetries"

# Check if the subfolder exists
if not os.path.exists(telemetries_path):
  # Create the subfolder
  os.makedirs(telemetries_path)

In [20]:
df_races = pd.DataFrame()
df_weather_data = pd.DataFrame()
df_races_control_data = pd.DataFrame()
df_drivers_info = pd.DataFrame()
df_telemetries = pd.DataFrame()
df_gears = pd.DataFrame()

Creating and storing dataframes following [1. Package exploration.ipynb](1.%20Package%20exploration.ipynb)

In [21]:
for i in range(1,23):   #Events 1 to 22 from 22 Formula 1 World Competition.
    session = fastf1.get_session(2022, i, 'R')
    session.load()

    df_race = session.laps   #Dataframe with info from each lap from each driver from each session (Event)
    df_race['LapTime(ms)'] = df_race['LapTime']//np.timedelta64(1, 'ms')   #Converts timedelta variable to float representing milliseconds
    df_race['EventName'] = session.event.EventName
    df_race['EventDate'] = session.event.EventDate
    df_races = pd.concat([df_races, df_race])

    session.weather_data['EventName'] = session.event.name   #Weather data for each event
    df_weather_data = pd.concat([df_weather_data,session.weather_data])

    session.race_control_messages['EventName'] = session.event.name  #Race control messages for each event
    df_races_control_data = pd.concat([df_races_control_data, session.race_control_messages])

    for driver in session.drivers:

        df_driver_info = pd.DataFrame()    
        
        for item in list(session.get_driver(driver).index):
            
            df_driver_info['EventName'] = pd.Series(session.event.EventName)       
            df_driver_info[item] = session.get_driver(driver)[item]
        
        df_drivers_info = pd.concat([df_drivers_info,df_driver_info])

        session_d = session.laps.pick_driver(driver)

        for lap in range(len(session_d)): #len(session_d)
            print("Race " + str(i) + " of 22" + " - " "Driver " + str(session.drivers.index(driver)+1) + " of "+ str(len(session.drivers)) +" - " + "Lap " + str(lap + 1) +" of " + str(len(session_d)))
            telemetry_lap = session_d.iloc[lap,].get_telemetry() 
            telemetry_lap['DriverNumber'] = driver
            telemetry_lap['LapNumber'] = lap + 1
            telemetry_lap['EventName'] = session.event.EventName
            event_telemetry_path = os.path.join(telemetries_path, session.event.EventName + ".xlsx")
            telemetry_lap.to_excel(event_telemetry_path)

            gear_lap = telemetry_lap[['nGear','LapNumber','DriverNumber','EventName','RelativeDistance']]
            gear_lap = gear_lap.copy()
            gear_lap['RelativeDistance'] = gear_lap['RelativeDistance'].diff()
            gear_lap = gear_lap.groupby(['nGear','LapNumber','DriverNumber','EventName']).sum().reset_index()
            df_gears = pd.concat([df_gears, gear_lap])

df_gears = df_gears.pivot(index=['DriverNumber','LapNumber','EventName'], columns = ['nGear'])
df_gears = df_gears.rename(columns={'RelativeDistance': "Gear"})
df_gears.columns = ['_'.join(str(s).strip() for s in col if s) for col in df_gears.columns]
df_gears.reset_index(inplace = True)

df_races = pd.merge(df_races,df_gears, how = 'left', on = ['DriverNumber','LapNumber','EventName'])

df_races.to_excel("./files/Races.xlsx")
df_weather_data.to_excel("./files/Weather.xlsx")
df_races_control_data.to_excel("./files/Control Messages.xlsx")
df_drivers_info.to_excel("./files/Driver Race Info.xlsx")


core           INFO 	Loading data for Bahrain Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '55', '44', '63', '20', '77', '31', '22', '14', '24', '47', '18', '23', '3', '4', '6', '27', '11', '1', '10']


Race 1 of 22 - Driver 1 of 20 - Lap 1 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 2 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 3 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 4 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 5 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 6 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 7 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 8 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 9 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 10 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 11 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 12 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 13 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 14 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 15 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 16 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 17 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 18 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 19 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 20 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 21 of 57
Race 1 of 22 - Driver 1 of 20 - Lap 22 of 57
Race 1 of 22 - Driv

core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
core        WARNING 	No lap data for driver 22
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 19 drivers: ['1', '16', '55', '11', '63', '31', '4', '10', '20', '44', '24', '27', '18', '23', '77', '14', '3', '6', '22']


Race 2 of 22 - Driver 1 of 19 - Lap 1 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 2 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 3 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 4 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 5 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 6 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 7 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 8 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 9 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 10 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 11 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 12 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 13 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 14 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 15 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 16 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 17 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 18 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 19 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 20 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 21 of 50
Race 2 of 22 - Driver 1 of 19 - Lap 22 of 50
Race 2 of 22 - Driv

core           INFO 	Loading data for Australian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '11', '63', '44', '4', '3', '31', '77', '10', '23', '24', '18', '47', '20', '22', '6', '14', '1', '5', '55']


...

Race 20 of 22 - Driver 1 of 20 - Lap 1 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 2 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 3 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 4 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 5 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 6 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 7 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 8 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 9 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 10 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 11 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 12 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 13 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 14 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 15 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 16 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 17 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 18 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 19 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 20 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 21 of 71
Race 20 of 22 - Driver 1 of 20 - Lap 22 of 

core           INFO 	Loading data for São Paulo Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['63', '44', '55', '16', '14', '1', '11', '31', '77', '18', '5', '24', '47', '10', '23', '6', '22', '4', '20', '3']


Race 21 of 22 - Driver 1 of 20 - Lap 1 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 2 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 3 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 4 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 5 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 6 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 7 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 8 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 9 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 10 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 11 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 12 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 13 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 14 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 15 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 16 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 17 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 18 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 19 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 20 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 21 of 71
Race 21 of 22 - Driver 1 of 20 - Lap 22 of 

core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '11', '55', '63', '4', '31', '18', '3', '5', '22', '24', '23', '10', '77', '47', '20', '44', '6', '14']


Race 22 of 22 - Driver 1 of 20 - Lap 1 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 2 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 3 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 4 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 5 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 6 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 7 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 8 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 9 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 10 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 11 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 12 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 13 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 14 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 15 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 16 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 17 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 18 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 19 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 20 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 21 of 58
Race 22 of 22 - Driver 1 of 20 - Lap 22 of 

In [2]:
from IPython.display import FileLink, FileLinks
FileLink('./1. Package exploration.ipynb')

c:\Users\PC\Github\Undergraduate-Final-Project\1. Package exploration.ipynb